In [ ]:
import pandas as pd
Electronics=pd.read_csv('ratings_Electronics.csv')
print(Electronics.head())
Electronics.columns=['user','item','rating','timestamp']
#print(Electronics.head())
del Electronics['timestamp']
#print(Electronics.head())

Videogames=pd.read_csv('ratings_Video_Games.csv')
Videogames.columns=['user','item','rating','timestamp']
#print(Videogames.head())
del Videogames['timestamp']
#print(Videogames.head())
data=Electronics.append(Videogames)
print(data.head())

In [ ]:
data.to_csv('Electronics_Video_Games.csv', index=False,header=False)

In [ ]:
import pyspark
import pandas as pd
import numpy as np
from pyspark import SparkContext, since
from pyspark.rdd import RDD
import pyspark.sql

from pyspark.mllib.recommendation import ALS
import math

In [ ]:
import os
from pyspark import SQLContext
from pyspark import SparkContext 
from pyspark.sql import SQLContext 
sc = pyspark.SparkContext('local')
sqlContext = SQLContext(sc)

In [ ]:
ratings = sc.textFile("Electronics_Video_Games.csv").map(lambda line: line.split(","))
ratings = ratings.toDF(['userID','itemID','rating'])
ratings.show()


In [ ]:
ratings1=ratings.select('userID').distinct()
ratings1.show()

In [ ]:
maxIndex = ratings1.count()
print(maxIndex)

In [ ]:
from pyspark.sql.functions import regexp_extract
from pyspark.sql.types import *
#ratings_filename="/Users/drakaris/Downloads/Electronics_Video_Games.csv"
#ratings_df_schema = StructType([StructField('userID', IntegerType()),StructField('movieID', IntegerType()),StructField('rating', DoubleType())])
from pyspark.sql import Row

In [ ]:
with_index = ratings1.rdd.zipWithIndex()
distinct_val = with_index.map(lambda x: (Row(userID = x[0][0], id =x[1])))
distinct_val = distinct_val.toDF()

In [ ]:
#ratings_df = sqlContext.read.format('com.databricks.spark.csv').options(header=True, inferSchema=False).schema(ratings_df_schema).load(ratings_filename)
#ratings_df = raw_ratings_df.drop('Timestamp')
#ratings_df.cache()
distinct_val.show()

In [ ]:
joined_df = ratings.join(distinct_val, ['userID'], "left")
rdd1 = joined_df.rdd

In [ ]:
rdd1.take(10)

In [ ]:
joined_df.show()

In [ ]:
ratings2=ratings.select('itemID').distinct()
ratings2.show()

In [ ]:
maxIndex = ratings2.count()
print(maxIndex)

In [ ]:
with_index = ratings2.rdd.zipWithIndex()
distinct_val1 = with_index.map(lambda x: (Row(itemID = x[0][0], id2 =x[1])))
distinct_val1 = distinct_val1.toDF()


In [ ]:
distinct_val1.show()

In [ ]:
joined_df1 = joined_df.join(distinct_val1, ['itemID'], "left")
rdd2 = joined_df1.rdd

In [ ]:
rdd2.take(10)

In [ ]:
joined_df1.show()

In [ ]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder


#als=ALS(userCol="userID",itemCol="movieID", ratingCol="rating", coldStartStrategy="drop",nonnegative=True)

In [ ]:
joined_df2=joined_df1.drop('itemID')
#joined_df2.show()

In [ ]:
joined_df3=joined_df2.drop('userID')
joined_df3.show()

In [ ]:
df=joined_df3.select("id","id2","rating")
df.show()


In [ ]:
#df.save('/Users/drakaris/Downloads/Electronics_Video_Games.csv', 'com.databricks.spark.csv')

In [ ]:
from pyspark.mllib.recommendation\
    import ALS,MatrixFactorizationModel, Rating
    
ratings = df.rdd.map(lambda x: Rating(int(x[0]),\
    int(x[1]), float(x[2])))    

train, test = ratings.randomSplit([0.8,0.2])

#print(train.count()) #70,005
#print(test.count()) #29,995



In [ ]:
#Need to cache the data to speed up training
train.cache()
#test.cache()

#Setting up the parameters for ALS
rank = 10 # Latent Factors to be made
numIterations = 15# Times to repeat process
regParam = 0.35
#Create the model on the training data
model = ALS.train(train, rank, numIterations, regParam)

In [ ]:
model.recommendUsers(242,10)

In [ ]:
# For User Y Find N Products to Promote
model.recommendProducts(196,10)

In [ ]:
#Predict Single Product for Single User
model.predict(196, 242)

In [ ]:
# Predict Multi Users and Multi Products
# Pre-Processing
pred_input = train.map(lambda x:(x[0],x[1]))   

In [ ]:
# Lots of Predictions
#Returns Ratings(user, item, prediction)
pred = model.predictAll(pred_input) 

In [ ]:
true_reorg = train.map(lambda x:((x[0],x[1]), x[2]))
pred_reorg = pred.map(lambda x:((x[0],x[1]), x[2]))

#Do the actual join
true_pred = true_reorg.join(pred_reorg)

#Need to be able to square root the Mean-Squared Error
from math import sqrt

MSE = true_pred.map(lambda r: (r[1][0] - r[1][1])**2).mean()
RMSE = sqrt(MSE)#Results in 0.7629908117414474

In [ ]:
print(RMSE)

In [ ]:
#Test Set Evaluation
#More dense, but nothing we haven't done before
test_input = test.map(lambda x:(x[0],x[1])) 
pred_test = model.predictAll(test_input)
test_reorg = test.map(lambda x:((x[0],x[1]), x[2]))
pred_reorg = pred_test.map(lambda x:((x[0],x[1]), x[2]))
test_pred = test_reorg.join(pred_reorg)
test_MSE = test_pred.map(lambda r: (r[1][0] - r[1][1])**2).mean()
test_RMSE = sqrt(test_MSE)#1.0145549956596238

In [ ]:
print(test_RMSE)

In [ ]:
#getting 0.1% of all the users to reccomend them top 3 items and calculate coverage
user_only=train.toDF()
users, smthelse=user_only.randomSplit([0.001,0.999])

In [ ]:
userlist=users.toPandas() 

In [ ]:
#reccomend each user 3 items
for row in userlist['user']: 
    a=model.recommendProducts(row,3)
    top_k_list.append(a)

In [ ]:
somelist=[]
recommended_number = 0
#getting total number of reccomended items
for row in range(1, len(top_k_list)): 
    #print(row)
    recommended_number += len(top_k_list[row])
    for a in range(0, len(top_k_list[row])):
        somelist.append(top_k_list[row][a].product)
        #print(len(top_k_list[row]))
   
    
    

In [ ]:
myset = set(somelist)

In [ ]:
len(myset)/recommended_number